In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from transformers import BertTokenizer, BertModel
import torch
import numpy as np
from sklearn.manifold import TSNE
import matplotlib
import matplotlib.pyplot as plt
from transformers import AutoTokenizer
from model.model.personalized_bert import EventBertModel
from model.model.PairwiseLecardPLM1 import PairwisePLM1
from formatter.PairwiseFormatter1 import PairwiseFormatter
from config_parser import create_config
from tools.init_tool1 import init_formatter
from torch.autograd import Variable
from lime.lime_text import LimeTextExplainer
import numpy as np
# plm_path = "/home/pub/fan/SCR/Pretrain_model/bert_base_chinese"
# tokenizer = AutoTokenizer.from_pretrained(plm_path)
device = torch.device("cuda")
plt.rcParams['font.sans-serif'] = ['SimHei']
matplotlib.rcParams['axes.unicode_minus'] = False  # 正确显示负号
def correct_state_dict(state_dict):
    """调整状态字典中的键名，以适应当前模型结构"""
    new_state_dict = {}
    for k, v in state_dict.items():
        # 移除所有不必要的'module.'前缀
        new_key = k.replace("module.", "")
        new_state_dict[new_key] = v
    return new_state_dict

def load_model(model_path):
    checkpoint = torch.load(model_path, map_location='cpu')
    model = PairwisePLM1()
    # 使用修正函数处理状态字典
    fixed_state_dict = correct_state_dict(checkpoint['model'])
    model.load_state_dict(fixed_state_dict)
    
    return model

# 假设你的模型保存路径为
model_path = '/hhd2/fan/SCR/output/PairwiseLecardBertBase_re/3.pth'
# model_path = '/hhd2/fan/SCR/output/PairwiseLecardSAILER_visual/3.pth'
model = load_model(model_path)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML
from lime.lime_text import LimeTextExplainer


PLM_vocab = "/hhd2/fan/SCR/Pretrain_model/bert_base_chinese"
tokenizer = AutoTokenizer.from_pretrained(PLM_vocab, trust_remote_code=True)
class ModelWrapper:
    def __init__(self, model, tokenizer, max_len, cls_id, sep_id, query_len, cand_len, use_event=True):
        self.model = model
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.cls_id = cls_id
        self.sep_id = sep_id
        self.query_len = query_len
        self.cand_len = cand_len
        self.use_event = use_event
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    def _preprocess_lime_text(self, texts):
        processed_data = []
        for text in texts:
            # 注意：这里的处理逻辑需要根据你的模型输入要求调整
            # 假设tokenizer能处理原始文本并返回input_ids, segment_ids, mask, 以及event_ids（如果use_event为True）
            tokenized = self.tokenizer.encode_plus(
                text,  # 这里可能需要更复杂的逻辑来处理原始数据结构
                max_length=self.max_len,
                padding='max_length',
                truncation=True,
                return_attention_mask=True,
                return_token_type_ids=self.use_event,  # 根据是否使用事件信息决定是否返回token_type_ids
            )
            input_ids = tokenized['input_ids']
            attention_mask = tokenized['attention_mask']
            segment_ids = tokenized.get('token_type_ids', [0]*len(input_ids))  # 如果模型不需要则默认全0
            
            # 如果有事件信息的处理逻辑，这里需要添加相应的代码
            if self.use_event:
                # 假设事件ID处理逻辑在这里
                event_ids = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 90, 90, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 14, 14, 0, 40, 40, 0, 0, 0, 0, 0, 0, 0, 0, 67, 67, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 69, 69, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 64, 64, 0, 0, 0, 67, 67, 0, 0, 0, 0, 67, 67, 0, 0, 0, 29, 0, 0, 0, 0, 0, 52, 52, 49, 49, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 67, 67, 0, 0, 0, 0, 0, 0, 44, 0, 0, 0, 28, 0, 0, 0, 0, 0, 0, 0, 0]  # 实现这部分逻辑以适应你的数据结构
                
            # 确保长度正确，可能需要根据你的实际逻辑调整
            while len(input_ids) < self.max_len:
                input_ids.append(0)
                attention_mask.append(0)
                segment_ids.append(0)
                if self.use_event:
                    event_ids.append(0)  # 假设添加的是事件ID的填充值
            
            processed_data.append({
                "input_ids": input_ids,
                "attention_mask": attention_mask,
                "segment_ids": segment_ids,
                "event_ids": event_ids if self.use_event else None,
            })
        
        return processed_data

    def predict(self, texts):
        preprocessed_data = self._preprocess_lime_text(texts)
        input_ids = torch.tensor([item["input_ids"] for item in preprocessed_data], dtype=torch.long, device=self.device)
        attention_mask = torch.tensor([item["attention_mask"] for item in preprocessed_data], dtype=torch.long, device=self.device)
        segment_ids = torch.tensor([item["segment_ids"] for item in preprocessed_data], dtype=torch.long, device=self.device)
        
        if self.use_event:
            event_ids = torch.tensor([item["event_ids"] for item in preprocessed_data if item["event_ids"] is not None], dtype=torch.long, device=self.device)
        else:
            event_ids = None
        
        with torch.no_grad():
            logits, _, _ = self.model(input_ids, attention_mask=attention_mask, token_type_ids=segment_ids, event_ids=event_ids)
            probs = torch.softmax(logits, dim=1)
        
        return probs.cpu().numpy()
explainer = LimeTextExplainer(class_names=['Relevance', 'Irrelevance'])

sample_text = "2013年，被告人陈益桃受被告人邓某某指使，为解决邓与被害人杨某某之间的经济纠纷，纠集被告人瞿某某等人强行将被害人杨某某带至他处看管。期间，被告人陈益桃等人采用殴打及言语威胁等方式，逼迫杨某某签下悔过书并承诺放弃经济利益。被告人陈益桃等人又逼迫被害人杨某某出具欠条支付辛苦费5万元。"

query_len = 138
cand_len =  0 
max_len = query_len + cand_len + 2
pad_id = tokenizer.pad_token_id
sep_id = tokenizer.sep_token_id
cls_id = tokenizer.cls_token_id
wrapper = ModelWrapper(model, tokenizer, max_len, cls_id, sep_id, query_len, cand_len, use_event=True)
exp = explainer.explain_instance(sample_text, wrapper.predict, num_features=6)

# # 显示结果
exp.show_in_notebook(text=sample_text)
# tokens = ['[CLS]', '2013', '年', '，', '被', '告', '人', '陈', '益', '桃', '受', '被', '告', '人', '邓', '某', '某', '指', '使', '，', '为', '解', '决', '邓', '与', '被', '害', '人', '杨', '某', '某', '之', '间', '的', '经', '济', '纠', '纷', '，', '纠', '集', '被', '告', '人', '瞿', '某', '某', '等', '人', '强', '行', '将', '被', '害', '人', '杨', '某', '某', '带', '至', '他', '处', '看', '管', '。', '期', '间', '，', '被', '告', '人', '陈', '益', '桃', '等', '人', '采', '用', '殴', '打', '及', '言', '语', '威', '胁', '等', '方', '式', '，', '逼', '迫', '杨', '某', '某', '签', '下', '悔', '过', '书', '并', '承', '诺', '放', '弃', '经', '济', '利', '益', '。', '被', '告', '人', '陈', '益', '桃', '等', '人', '又', '逼', '迫', '被', '害', '人', '杨', '某', '某', '出', '具', '欠', '条', '支', '付', '辛', '苦', '费', '5', '万', '元', '。', '[SEP]']
# print(len(tokens))